In [50]:
import pandas as pd
import os
import sklearn
import pymc3 as pm

In [39]:
# Import data
path_to_data = '../data/ml-32m/'
print(path_to_data)
ratings = pd.read_csv(os.path.join(path_to_data, 'ratings.csv'))
movies = pd.read_csv(os.path.join(path_to_data, 'movies.csv'))

../data/ml-32m/


In [41]:
print(ratings.head(3), '\n')
print(movies.head(3))

   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976 

   movieId                    title  \
0        1         Toy Story (1995)   
1        2           Jumanji (1995)   
2        3  Grumpier Old Men (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  


In [43]:
print(ratings.describe())

             userId       movieId        rating     timestamp
count  3.200020e+07  3.200020e+07  3.200020e+07  3.200020e+07
mean   1.002785e+05  2.931861e+04  3.540396e+00  1.275241e+09
std    5.794905e+04  5.095816e+04  1.058986e+00  2.561630e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    5.005300e+04  1.233000e+03  3.000000e+00  1.051012e+09
50%    1.002970e+05  3.452000e+03  3.500000e+00  1.272622e+09
75%    1.504510e+05  4.419900e+04  4.000000e+00  1.503158e+09
max    2.009480e+05  2.927570e+05  5.000000e+00  1.697164e+09


In [44]:
# Check for missing values
print(ratings.isnull().sum())
print(movies.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


In [36]:
# Normalise data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
ratings['scaled_rating'] = scaler.fit_transform(ratings[['rating']])

In [46]:
# Merge dataframes to make dataframe containing ratings and genres
full_ratings = pd.merge(ratings, movies, on='movieId')

    userId  movieId  rating   timestamp                         title  \
0        1       17     4.0   944249077  Sense and Sensibility (1995)   
1        3       17     5.0  1084485217  Sense and Sensibility (1995)   
2       15       17     4.5  1289858271  Sense and Sensibility (1995)   
3       28       17     4.0   961513829  Sense and Sensibility (1995)   
4       29       17     4.0   845056111  Sense and Sensibility (1995)   
5       32       17     5.0   837622358  Sense and Sensibility (1995)   
6       39       17     2.0   859534406  Sense and Sensibility (1995)   
7       40       17     5.0   940027447  Sense and Sensibility (1995)   
8       43       17     5.0   834322652  Sense and Sensibility (1995)   
9       57       17     3.0   852581231  Sense and Sensibility (1995)   
10      59       17     4.0  1030398472  Sense and Sensibility (1995)   
11      62       17     4.0   916456163  Sense and Sensibility (1995)   
12      78       17     5.0   901987955  Sense and 